## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Tensorflow
import tensorflow as tf


In [ ]:
df = application_df

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
df = df.drop(columns=['EIN', 'NAME'])

In [ ]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
unique_values = df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [ ]:
df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
application_type_counts = df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

types_to_replace = application_type_counts[application_type_counts < 200].index
# df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(types_to_replace, 'Other')
df['AFFILIATION_TYPE'] = df['APPLICATION_TYPE'].replace(types_to_replace, 'Other')

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 200
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
#  YOUR CODE GOES HERE
# Choose a cutoff value
cutoff_value = 200

# Look at CLASSIFICATION value counts
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications that occur less than the cutoff value
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for classification in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classification, "Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()


,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
Other,1197
C7000,777
C1700,287


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 200
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
Other,1197
C7000,777
C1700,287


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
# Create dummy variables for the specified categorical columns
application_dummies = pd.get_dummies(application_df[['APPLICATION_TYPE', 'CLASSIFICATION']], drop_first=True)

# Concatenate the dummy variables with the original DataFrame
application_df = pd.concat([application_df, application_dummies], axis=1)

# Optionally, drop the original categorical columns if they are no longer needed
application_df.drop(['APPLICATION_TYPE', 'CLASSIFICATION'], axis=1, inplace=True)

# Check the updated DataFrame
application_df.head()

,EIN,NAME,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,CLASSIFICATION_C1200,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,Independent,ProductDev,Association,1,0,N,5000,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,CompanySponsored,ProductDev,Association,1,0,N,5000,0,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
application_df.columns

Index(['EIN', 'NAME', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'STATUS',
       'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C1700', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100',
       'CLASSIFICATION_C3000', 'CLASSIFICATION_C7000', 'CLASSIFICATION_Other'],
      dtype='object')

In [ ]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)  # Features
y = application_df['IS_SUCCESSFUL']  # Target

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, test_size=0.2)  # Adjust test_size as needed

# Check the shapes of the resulting datasets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(X_train)


(27439, 24) (6860, 24) (27439,) (6860,)
             EIN                                               NAME  \
33580  946102210                                    AMERICAN LEGION   
21025  562559152  EXPERIMENTAL AIRCRAFT ASSOCIATION ROCKY MOUNTA...   
20933  562444266                     FOURTH DISTRICT DENTAL SOCIETY   
10861  364687225                 COUNTRY DANCE AND SONG SOCIETY INC   
28821  813585813                                 TEMPLE LIFE CENTER   
...          ...                                                ...   
20558  510649064                                       SONGEAS KIDS   
18547  473147918                   TOGETHER WE CAN CHANGE THE WORLD   
3476   237112123                      AMERICAN POSTAL WORKERS UNION   
22633  621624993  NATIONAL SOCIETY OF THE DAUGHTERS OF THE AMERI...   
16854  465014769        REGIONAL FOOD BANK OF NORTHEAST FLORIDA INC   

            AFFILIATION      USE_CASE ORGANIZATION  STATUS     INCOME_AMT  \
33580  CompanySponsored  Prese

In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)  # Features
y = application_df['IS_SUCCESSFUL']  # Target

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, test_size=0.2)  # Adjust test_size as needed

# Check if any columns are of type 'object'
object_columns = X_train.select_dtypes(include=['object']).columns

# If there are object columns, drop them before scaling
if len(object_columns) > 0:
    X_train = X_train.drop(object_columns, axis=1)
    X_test = X_test.drop(object_columns, axis=1)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Check the shapes of the resulting datasets
print(X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape)
print(X_train_scaled)

(27439, 18) (6860, 18) (27439,) (6860,)
[[ 1.73900538  0.01350021 -0.03369006 ... -0.2430079  -0.15078657
  -0.18906782]
 [ 0.17439341  0.01350021 -0.03369006 ... -0.2430079  -0.15078657
  -0.18906782]
 [ 0.17392475  0.01350021 -0.03283533 ... -0.2430079  -0.15078657
  -0.18906782]
 ...
 [-1.15322367  0.01350021 -0.03246445 ...  4.11509258 -0.15078657
  -0.18906782]
 [ 0.41534451  0.01350021 -0.03369006 ... -0.2430079  -0.15078657
  -0.18906782]
 [-0.22352569  0.01350021 -0.01047127 ... -0.2430079  -0.15078657
  -0.18906782]]


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │           1,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,981 (15.55 KB)

 Trainable params: 3,981 (15.55 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5937 - loss: 0.6559
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6179 - loss: 0.6415
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6182 - loss: 0.6374
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6222 - loss: 0.6374
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6184 - loss: 0.6372
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6203 - loss: 0.6366
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6252 - loss: 0.6319
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6194 - loss: 0.6336
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6189 - loss: 0.6363
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6224 - loss: 0.6323
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6225 - loss: 0.6331
Epoch 12/100
858/858 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.6430 - loss: 0.6224
Loss: 0.6223610043525696, Accuracy: 0.6430029273033142


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save('AlphabetSoupCharity.h5')